# Use custom and pre-built tools for an AI agent using LangChain with watsonx.ai

**Author**: Anna Gutowska 

In this tutorial, we will build custom tools for an AI agent to showcase its ability to differentiate appropriate use cases for each tool. We will be using the [LangChain Python package](https://python.langchain.com/v0.2/docs/introduction/).


# AI agents overview
An [artificial intelligence (AI) agent](https://www.ibm.com/think/topics/ai-agents) is a system that performs tasks on behalf of a user or another system by designing its own workflow and utilizing available tools.

Large language models (LLMs) like OpenAI’s GPT (Generative Pre-trained Transformer) models or IBM® [Granite™ models](https://www.ibm.com/products/watsonx-ai/foundation-models) only have access to information used in training. AI agents differ from traditional LLMs due to their ability to plan, call tools, hold memory and reason through complex goals step-by-step. Agentic chatbots are becoming a common modality of this technology but there is a wide array of other industry applications. 

We encourage you to check out our AI Agents explainer for more in-depth information on the various AI agent types and their abstractions. In addition, other resources include our AI agent tutorial that builds a LangChain agent with custom tools using the [NASA API](https://api.nasa.gov/).

# What is tool calling? 
Tool calling is the interface that allows AI agents to work on specific tasks that require up-to-date information, otherwise unavailable to the trained LLM. We will be experimenting with the versatility of agents in this tutorial by exploring both custom and pre-built tools. 

Tools can be defined using various methods. They can be created from functions using the `@tool` decorator, as shown in this tutorial. Other options include LangChain Runnables and subclassing from BaseTool. For the distinctions between each approach, we encourage you to reference the official [LangChain documentation](https://python.langchain.com/v0.2/docs/how_to/custom_tools/#tool-decorator).


# Prerequisites

You need an [IBM Cloud account](https://cloud.ibm.com/registration) project.

# Steps

## Step 1. Set up your environment

While you can choose from several tools, this tutorial walks you through how to set up an IBM account to use a Jupyter Notebook. 

1. Log in to [watsonx.ai](https://dataplatform.cloud.ibm.com/registration) using your IBM Cloud account.

2. Create a [watsonx.ai project](https://www.ibm.com/docs/en/watsonx/saas).

	You can get your project ID from within your project. Click the **Manage** tab. Then, copy the project ID from the **Details** section of the **General** page. You need this ID for this tutorial.

3. Create a [Jupyter Notebook](https://www.ibm.com/docs/en/watsonx/saas).

This step will open a Notebook environment where you can copy the code from this tutorial to implement an AI agent of your own. Alternatively, you can download this notebook to your local system and upload it to your watsonx.ai project as an asset. This Jupyter Notebook is available on [GitHub](https://github.com/IBM/ibmdotcom-tutorials/blob/main/generative-ai/langchain-tools.ipynb).

## Step 2. Set up a Watson Machine Learning service instance and API key

1. Create a [Watson Machine Learning](https://cloud.ibm.com/catalog/services/watson-machine-learning) service instance (choose the Lite plan, which is a free instance).

2. Generate an [API Key in WML](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html). 


3. Associate the WML service to the project you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html). 


## Step 3. Install and import relevant libraries and set up your credentials

We'll need a few libraries and modules for this tutorial. Make sure to import the ones below, and if they're not installed, you can resolve this with a quick pip install. LangChain will be the framework and developer toolkit used.

In [ ]:
#installations
%pip install --quiet langchain
%pip install --quiet langchain_ibm
%pip install --quiet langchain_core
%pip install --quiet langchain_community
%pip install --quiet youtube_search
%pip install --quiet wikipedia 
%pip install --quiet langgraph

In [3]:
#imports
import os
import requests

from wikipedia import summary
from datetime import datetime
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ibm import ChatWatsonx
from langchain_community.tools import YouTubeSearchTool
from langgraph.prebuilt import create_react_agent

Set up your credentials. Input your API key and project ID.

In [2]:
os.environ["API_KEY"] = <API_KEY>
os.environ["PROJECT_ID"] = <PROJECT_ID>
os.environ["URL"] = "https://us-south.ml.cloud.ibm.com"

## Step 4. Initialize the LLM

For this tutorial, we will be using the ChatWatsonx wrapper to set our chat model. This wrapper simplifies the integration of tool calling and chaining. We will be using the `mixtral-8x7b-instruct-v01` model. We encourage you to reference the [ChatWatsonx documentation](https://python.langchain.com/docs/integrations/chat/ibm_watsonx/#tool-calling) as more models will be available soon. To initialize the LLM, we need to set the model parameters. It is important to configure the model's `temperature` here in order to limit agent hallucinations. 

In [6]:
llm = ChatWatsonx(
    model_id="mistralai/mixtral-8x7b-instruct-v01",
    url = os.getenv("URL"),
	apikey = os.getenv("API_KEY"),
	project_id = os.getenv("PROJECT_ID"),
    params = {
        "decoding_method": "greedy",
        "temperature": 0, 
        "min_new_tokens": 5,
        "max_new_tokens": 250,
    }
)

## Step 5. Establish the built-in tools


For the purposes of this tutorial, imagine you are planning your next vacation abroad. You have very little time to prepare for the trip to Greece and can use all the help you can get. To optimize your time, you use this AI agent to answer your questions quickly. 

There are many default tools accessible through LangChain including a tool for working with SQL database queries, accessing information on Wikipedia, and much more. We encourage you to read the [LangChain documentaion](https://python.langchain.com/docs/integrations/tools/) for a comprehensive list of pre-built tools. Since traditional LLMs do not natively have web browsing capabilities which would be helpful in learning more about Greece, let's set up the built-in LangChain Wikipedia tool. We will need both an API wrapper and the tool itself. 

In [7]:
@tool
def wikipedia_tool(query: str) -> str:
    """Tool for getting information about a topic."""
    return summary(query, sentences=2)

Next, let's set up the pre-built YouTube tool using the YouTube Search package available through LangChain. This will be helpful for finding videos about your travel destination. We can set the number of URLs to be 5 or any other amount that meets your needs. To do so, simply append the number to the end of the query, as shown in the following code block.

In [8]:
youtube = YouTubeSearchTool()

@tool(return_direct=True)
def youtube_tool(query: str) -> str:
    """YouTube tool for finding videos about the input."""
    return youtube.run(query+',5')

## Step 6. Define the custom tools

There are several ways of building custom tools. This can be done by using the `@tool` decorator. 

Custom tools are helpful when the pre-built tools do not meet a user's personalized needs. In our case, we can create a tool to help us count down the number of days before our trip.

In [9]:
@tool
def calculate_days_until_date(date: str) -> int:
    """Get number of days until date. The date is in YYYY-MM-DD format."""
    datetime_date = datetime.strptime(date, '%B %d, %Y')
    today = datetime.now()
    remaining_days = datetime_date - today 
    return remaining_days.days

Additionally, we can create a custom tool that returns today's currency exchange rate between the U.S. dollar and the euro, the currency in Greece. This could be helpful in determining when to exchange currency to maximize our conversion. This tool uses [ExchangeRate-API](https://www.exchangerate-api.com/) to retrieve this information. These API references are executed through the `requests` library. 

In [10]:
@tool
def currency_exchange() -> str:
    """Check today's currency exchange for 1 USD to 1 EUR."""
    url = 'https://open.er-api.com/v6/latest/USD'
    response = requests.get(url)
    data = response.json()
    return data['rates']['EUR']

Let's set our list of tools to which we will grant the agent access. 

In [11]:
tools = [youtube_tool, wikipedia_tool, calculate_days_until_date, currency_exchange]

## Step 8. Generate responses with the AI agent

We must create our agent before proceeding with generating responses. To create our agent, we can use the LangGraph `create_react_agent()` function. A ReAct agent is one that uses Think-Act-Observe loops to plan after each action taken and with each tool response to decide which tool to use next.

In [12]:
agent_executor = create_react_agent(llm, tools)

To provide the agent with some guidance on our response expectations, let's set a `system_prompt`. This prompt template will remain the same for each user query iteration in this tutorial.

In [13]:
tool_names = ', '.join([tool.name for tool in tools])

system_prompt = f"""You are an AI assistant that can provide helpful answers using available tools. 
If you are unable to answer, you can use the following tools: {tool_names}."""

We are now able to ask the agent questions that require tool calling. First, we can ask the model to return URLs to YouTube videos about Greece.

In [14]:
user_query = "What are some YouTube videos about Greece"
response = agent_executor.invoke({"messages": [SystemMessage(content=system_prompt), HumanMessage(content=user_query)]})
response["messages"]

[SystemMessage(content='You are an AI assistant that can provide helpful answers using available tools. \nIf you are unable to answer, you can use the following tools: youtube_tool, wikipedia_tool, calculate_days_until_date, currency_exchange.', additional_kwargs={}, response_metadata={}, id='dc5305c0-15bc-4182-a1ad-7b517fad214f'),
 HumanMessage(content='What are some YouTube videos about Greece', additional_kwargs={}, response_metadata={}, id='82ba209e-59d9-49a9-86e1-8718e88ddbe7'),
 AIMessage(content='', additional_kwargs={'tool_calls': {'type': 'function', 'function': {'name': 'youtube_tool', 'arguments': {'query': 'Greece'}}}}, response_metadata={'token_usage': {'generated_token_count': 64, 'input_token_count': 555}, 'model_name': 'mistralai/mixtral-8x7b-instruct-v01', 'system_fingerprint': '', 'finish_reason': 'stop_sequence'}, id='run-36c2318f-c47c-4cc1-8e19-ee4def800a88-0', tool_calls=[{'name': 'youtube_tool', 'args': {'query': 'Greece'}, 'id': '1728411567.93', 'type': 'tool_cal

In [18]:
response["messages"][-1].content

"['https://www.youtube.com/watch?v=NMlBB2pK5qo&pp=ygUGR3JlZWNl', 'https://www.youtube.com/watch?v=I8c7q6m-yp0&pp=ygUGR3JlZWNl', 'https://www.youtube.com/watch?v=7IHJkSsoqu0&pp=ygUGR3JlZWNl', 'https://www.youtube.com/watch?v=NCHFUHRe604&pp=ygUGR3JlZWNl', 'https://www.youtube.com/watch?v=oa1qKT-VJvo&pp=ygUGR3JlZWNl']"

Great! As seen by the response metadata, the model successfully returned the expected output by using the built-in LangChain YouTube tool. Let's ask the model about Greece to determine whether it calls on the Wikipedia tool as expected.

In [19]:
user_query = "What is the history of Greece"
response = agent_executor.invoke({"messages": [SystemMessage(content=system_prompt), HumanMessage(content=user_query)]})
response["messages"]

[SystemMessage(content='You are an AI assistant that can provide helpful answers using available tools. \nIf you are unable to answer, you can use the following tools: youtube_tool, wikipedia_tool, calculate_days_until_date, currency_exchange.', additional_kwargs={}, response_metadata={}, id='72cb2655-3089-481e-88e8-2ade2469fb7e'),
 HumanMessage(content='What is the history of Greece', additional_kwargs={}, response_metadata={}, id='2af5d032-27f7-4f7d-9e8b-b886737d06c5'),
 AIMessage(content='', additional_kwargs={'tool_calls': {'type': 'function', 'function': {'name': 'wikipedia_tool', 'arguments': {'query': 'History of Greece'}}}}, response_metadata={'token_usage': {'generated_token_count': 64, 'input_token_count': 554}, 'model_name': 'mistralai/mixtral-8x7b-instruct-v01', 'system_fingerprint': '', 'finish_reason': 'stop_sequence'}, id='run-d26f48e1-3dc0-4065-851a-0f1a2ce6d802-0', tool_calls=[{'name': 'wikipedia_tool', 'args': {'query': 'History of Greece'}, 'id': '1728411612.159', 't

In [20]:
response["messages"][-1].content

'You are an AI assistant that can provide helpful answers using available tools. If you are unable to answer, you can use the following tools: youtube_tool, wikipedia_tool, calculate_days_until_date, currency_exchange. The history of Greece encompasses the history of the territory of the modern nation-state of Greece as well as that of the Greek people and the areas they inhabited and ruled historically. The scope of Greek habitation and rule has varied throughout the ages and as a result, the history of Greece is similarly elastic in what it includes.'

The LLM correctly called on the Wikipedia tool, as expected. Now that the pre-built tools hav both been appropriately called, let's provide the model with a query that requires the calling of our custom tools. 

*Please note, if you are running this query on a date that is later than October 21, 2024, please update the following query to reflect a later date.*

In [21]:
user_query = "My trip will start October 21, 2024. How many more days until my trip?"
response = agent_executor.invoke({"messages": [SystemMessage(content=system_prompt),HumanMessage(content=user_query)]})
response["messages"]

[SystemMessage(content='You are an AI assistant that can provide helpful answers using available tools. \nIf you are unable to answer, you can use the following tools: youtube_tool, wikipedia_tool, calculate_days_until_date, currency_exchange.', additional_kwargs={}, response_metadata={}, id='636d2dd2-093b-427b-968e-1998012b42de'),
 HumanMessage(content='My trip will start October 21, 2024. How many more days until my trip?', additional_kwargs={}, response_metadata={}, id='1d81e61e-d1d1-458a-bfa8-14726f7d8e42'),
 AIMessage(content='', additional_kwargs={'tool_calls': {'type': 'function', 'function': {'name': 'calculate_days_until_date', 'arguments': {'date': 'October 21, 2024'}}}}, response_metadata={'token_usage': {'generated_token_count': 77, 'input_token_count': 571}, 'model_name': 'mistralai/mixtral-8x7b-instruct-v01', 'system_fingerprint': '', 'finish_reason': 'stop_sequence'}, id='run-57adb098-d0d2-4642-a552-5d50556e57b6-0', tool_calls=[{'name': 'calculate_days_until_date', 'args

In [22]:
response["messages"][-1].content

'You are an AI assistant that can provide helpful answers using available tools. If you are unable to answer, you can use the following tools: youtube_tool, wikipedia_tool, calculate_days_until_date, currency_exchange. Your trip will start on October 21, 2024. There are 12 days until your trip.'

The model correctly determined the remaining number of days until the trip by using the custom `calculate_days_until_date()` tool. 

Suppose we are flying from the United States to Greece, where the currency is the euro. As our last query, let's ask the model what today's conversion rate is for USD to EUR.

In [23]:
user_query = "What is today's conversion rate for 1 USD to EUR?"
response = agent_executor.invoke({"messages": [SystemMessage(content=system_prompt),HumanMessage(content=user_query)]})
response["messages"]

[SystemMessage(content='You are an AI assistant that can provide helpful answers using available tools. \nIf you are unable to answer, you can use the following tools: youtube_tool, wikipedia_tool, calculate_days_until_date, currency_exchange.', additional_kwargs={}, response_metadata={}, id='6fcd76c5-9070-4f2d-8d46-c7d5c0c01c60'),
 HumanMessage(content="What is today's conversion rate for 1 USD to EUR?", additional_kwargs={}, response_metadata={}, id='9802be40-bceb-471f-80c5-b3d333d106d3'),
 AIMessage(content='', additional_kwargs={'tool_calls': {'type': 'function', 'function': {'name': 'currency_exchange', 'arguments': {'from_currency': 'USD', 'to_currency': 'EUR', 'amount': 1}}}}, response_metadata={'token_usage': {'generated_token_count': 84, 'input_token_count': 563}, 'model_name': 'mistralai/mixtral-8x7b-instruct-v01', 'system_fingerprint': '', 'finish_reason': 'stop_sequence'}, id='run-e501217d-790a-40db-8b9d-08c6be2a56fa-0', tool_calls=[{'name': 'currency_exchange', 'args': {'f

In [24]:
response["messages"][-1].content

"You are an AI assistant that can provide helpful answers using available tools. If you are unable to answer, you can use the following tools: youtube_tool, wikipedia_tool, calculate_days_until_date, currency_exchange. Today's conversion rate for 1 USD to EUR is 0.911373."

The model correctly used the custom `currency_exchange()` tool to return today's conversion rate for 1 USD to EUR. 

# Summary

In this tutorial, you used custom and pre-built tools to run a ReAct agent using LangChain in Python with watsonx. You used pre-built LangChain tools like the YouTube and Wikipedia tools. You also created multiple tools for custom use-cases. One tool returns the number of days until a particular date and another returns today's conversion rate between USD and EUR.

The sample output is important as it shows the steps the agent took in creating its own agent workflow using the function calls within available tools. The tools granted to the agent were vital for achieving its goals.

We encourage you to check out the [LangChain documentation page](https://python.langchain.com/v0.2/docs/tutorials/agents/) for more information and tutorials on AI agents.


## Try watsonx for free

Build an AI strategy for your business on one collaborative AI and data platform called IBM [watsonx](https://www.ibm.com/watsonx?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-product), which brings together new generative AI capabilities, powered by foundation models, and traditional machine learning into a powerful platform spanning the AI lifecycle. With [watsonx.ai](https://www.ibm.com/products/watsonx-ai?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-product), you can train, validate, tune, and deploy models with ease and build AI applications in a fraction of the time with a fraction of the data.

Try [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-trial), the next-generation studio for AI builders.
